In [ ]:
#players
players_drop = ['birthday', 'birthday_GMT', 'league', 'season', 'nationality']

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'players' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                #average_rating_overall 이후 컬럼 제거
                if 'average_rating_overall' in df.columns:
                    cut_idx = df.columns.get_loc('average_rating_overall')
                    df = df.iloc[:, :cut_idx+1]
                    
                df_processed = df.drop(columns=[col for col in players_drop if col in df.columns])
                    
                if 'Current Club' in df_processed.columns:
                    df_processed = df_processed.rename(columns={'Current Club': 'team_name'})
                    
                new_path = os.path.join(root, file.replace('.csv', '_processed.csv'))
                df_processed.to_csv(new_path, index=False)
                print(f"[완료] players 컬럼 제거 후 저장: {new_path}")
                
            except Exception as e:
                print(f"[오류] {file_path}: {e}")


In [ ]:
#teams

#teams+teams2 병합
for root, dirs, files in os.walk(data_root):
    files_lower = [f.lower() for f in files]
    
    #teams와 teams2 파일이 모두 존재하는 폴더만 처리
    if any('teams' in f for f in files_lower) and any('teams2' in f for f in files_lower):
        try:
            #실제 파일명 찾기
            teams1_file = next(f for f in files if 'teams' in f.lower())
            teams2_file = next(f for f in files if 'teams2' in f.lower())
            
            #파일 경로
            path1 = os.path.join(root, teams1_file)
            path2 = os.path.join(root, teams2_file)
            
            #파일 불러오기
            teams1 = pd.read_csv(path1)
            teams2 = pd.read_csv(path2)
            
            #병합
            teams = pd.merge(teams1, teams2, on='team_name', suffixes=('_t1', '_t2'))
            
            #중복 컬럼 제거
            cols_to_drop = []
            for col in teams.columns:
                if col.endswith('_t1'):
                    base = col[:-3]
                    t2_col = base + '_t2'
                    if t2_col in teams.columns and teams[col].equals(teams[t2_col]):
                        cols_to_drop.append(t2_col)
                        
            teams_clean = teams.drop(columns=cols_to_drop)
            teams_clean.columns = [c.replace('_t1','').replace('_t2','') for c in teams_clean.columns]
            
            #저장 경로
            merged_filename = teams1_file.replace('teams', 'merged_teams')
            save_path = os.path.join(root, merged_filename.replace('.csv', '_processed.csv'))
            teams_clean.to_csv(save_path, index=False)
            print(f"[완료] 병합 및 저장: {save_path}")
        except Exception as e:
            print(f"[오류] {root}: {e}")
       
#삭제     
teams_drop = ['team_name', 'season', 'country']
    
for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'merged_teams' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                df = df.drop(columns=[col for col in teams_drop if col in df.columns])
                
                if 'common_name' in df.columns:
                    df = df.rename(columns={'common_name':'team_name'})
                    df.to_csv(file_path, index=False)
                    print(f"[완료] teams: {file_path}")
                else:
                    print(f"[오류] common_name 변경:{file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")